<a href="https://colab.research.google.com/github/kghilothia06/Time-Series-Forecasting/blob/main/05_grid_search_SARIMA_car_sales_13_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Libraries

In [1]:
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from pandas import read_excel

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Function for one-step sarima forecast

In [2]:
def sarima_forecast(history, config):
	order, sorder, trend = config
	# define model
	model = SARIMAX(history, order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
	# fit model
	model_fit = model.fit(disp=False)
	# make one step forecast
	yhat = model_fit.predict(len(history), len(history))
	return yhat[0]

Function for walk-forward validation for univariate data

In [3]:
def walk_forward_validation(data,n_test,cfg):
  predictions = list()
  # split dataset
  train = data[:-n_test]
  test = data[-n_test:] 
  # seed history with training dataset
  history = [x for x in train]
  # step over each time-step in the test set
  for i in range(len(test)):
    # fit model and make forecast for history
    yhat = sarima_forecast(history, cfg)
    # store forecast in list of predictions
    predictions.append(yhat)
    # add actual observation to history for the next loop
    history.append(test[i])
  # estimate prediction error
  error = sqrt(mean_squared_error(test, predictions))
  print('RMSE : ' + str(error))

load dataset for monthly sales prediction



In [4]:
series = read_excel('/content/Sales-monthly.xlsx', header=0, index_col=0)
data = series.values
# data split
n_test = 12

specify model configs

In [5]:
p = 2
d = 1
q = 0 
P = 2
D = 0
Q = 0
m = 12
t = 'n'

cfg = [(p,d,q), (P,D,Q,m), t]

In [6]:
walk_forward_validation(data,n_test,cfg)

RMSE : 29052.927267558578


load dataset for weekly sales prediction



In [7]:
series = read_excel('/content/Sales-weekly.xlsx', header=0, index_col=0)
data = series.values
# data split
n_test = 12

specify model configs

In [8]:
p = 2
d = 1
q = 0
P = 2
D = 0
Q = 0
m = 12
t = 'n'

cfg = [(p,d,q), (P,D,Q,m), t]

In [9]:
walk_forward_validation(data,n_test,cfg)

RMSE : 47878.55481780116
